# Unit 2: Set Outdoors Navigation

Here you will learn how to access the GPS data and use it to navigate around without a mapping system. You will have to relay upon the lasers to detect any obstacles and navigate around them to get to your destination in GPS coordinates.<br>
You will also learn how to visualise real maps in RVIZ based on GPS data.

## GPS basic concepts

So you have your Summit Robot navigating with your map, thats great. But what happens when you want to move ouside your map? How do you localise yourself without AMCL system?<br>
Well the answer is to use the **GPS data**.<br>

It seems simple right? Well yes and no. GPS data has some limitations:<br>
* GPS data is not very precise: GPS data oscilates in value and your position will move a lot. This is anavoidable and you will have to mitigate that error in your own way. **Odometry** data will mitigate some of these errors but in case of Summit XL, because it moves by sliding across, the odometry can have quite a lot of error. This is because odometry normaly relies on the fact that the wheels dont slide on the floor and there is 1:1 ratio of movement. This is not the case when you lose traction.  Bare that in mind.
* GPS gives you no orientation: GPS only gives you a point in space, no orientation. This means that you need to complement that with a **compas** of some kind. Here is where **IMU** comes in handy.


With all this in mind, you will be able to have a crude but usefull system to position and move your robot around an outdoors terrain without map.<br>
Of course, you can move around with GPS and while doing it, generate a new map, making bigger the area where map localization ( much more precise ) can be used next time.

## Get The GPS navigation running

### Step 1: Disconnect AMCL

Of course the first thing you have to do is disconnect the localization based on map, because now it wont be reliable when you get out of the mapped area. So you will have to comment the launch of AMCL

**start_movebase_new.launch**

In [ ]:
<?xml version="1.0"?>
<!-- NEW SUMIT XL NAVIGATION -->
<launch>
        
    <!-- Run the map server -->
    <include file="$(find sumit_xl_tools)/launch/start_map_server.launch" />
    <!--- Run AMCL -->
    <!--
    <include file="$(find sumit_xl_tools)/launch/start_localization.launch" />
    -->
    <remap from="cmd_vel" to="/move_base/cmd_vel" />
    
    <arg name="no_static_map" default="false"/>
    
    <arg name="base_global_planner" default="navfn/NavfnROS"/>
    <arg name="base_local_planner" default="dwa_local_planner/DWAPlannerROS"/>
    <!-- <arg name="base_local_planner" default="base_local_planner/TrajectoryPlannerROS"/> -->
    
    <node pkg="move_base" type="move_base" respawn="false" name="move_base" output="screen">
        
        <param name="base_global_planner" value="$(arg base_global_planner)"/>
        <param name="base_local_planner" value="$(arg base_local_planner)"/>  
        <rosparam file="$(find sumit_xl_tools)/new_config/planner.yaml" command="load"/>
        
        <!-- observation sources located in costmap_common.yaml -->
        <rosparam file="$(find sumit_xl_tools)/new_config/costmap_common.yaml" command="load" ns="global_costmap" />
        <rosparam file="$(find sumit_xl_tools)/new_config/costmap_common.yaml" command="load" ns="local_costmap" />
        
        <!-- local costmap, needs size -->
        <rosparam file="$(find sumit_xl_tools)/new_config/costmap_local.yaml" command="load" ns="local_costmap" />
        <param name="local_costmap/width" value="5.0"/>
        <param name="local_costmap/height" value="5.0"/>
        
        <!-- static global costmap, static map provides size -->
        <rosparam file="$(find sumit_xl_tools)/new_config/costmap_global_static.yaml" command="load" ns="global_costmap" unless="$(arg no_static_map)"/>
        
        <!-- global costmap with laser, for odom_navigation_demo -->
        <rosparam file="$(find sumit_xl_tools)/new_config/costmap_global_laser.yaml" command="load" ns="global_costmap" if="$(arg no_static_map)"/>
        <param name="global_costmap/width" value="100.0" if="$(arg no_static_map)"/>
        <param name="global_costmap/height" value="100.0" if="$(arg no_static_map)"/>
    </node>

</launch>

This should be your new move_move launch system. As you can see you still launch the map server, but you comment the launch **start_localization.launch** that launches the amcl.<br>
The reason you leave the map server is that move base needs a map to plan all its paths.<br>
So the next step will be to adapt your old map to a new one that can be used in a larger area.

### Upgrate your old map to have a huge free area outside the mapped area

So this could be your old map image:

<img src="img/old_clean_map.png" width="600"></img>

As you can see its a huge image but only a small part is white ( free). You have to change that, make it all the grey ( unknown ) area, **free**.<br>
For that you can use any software that can biarise an image and turn the grey colour to white. An exaple is this page: https://manytools.org/image/colorize-filter/. You can select the **DualTone** and maximise the contrast to get an image like thisone:

<img src="img/new_clean_map.png" width="600"></img>

Nice. Save it as **new_clean_map.pgm** and create a new yaml file based on the old one. Because the image is the exact same size and has the same features, you just have to make a copy.<br>

In [ ]:
image: /absolute_path_to_your_file/new_clean_map.pgm
resolution: 0.050000
origin: [-50.000000, -50.000000, 0.000000]
negate: 0
occupied_thresh: 0.65
free_thresh: 0.196


Now that you have the new image, change that in the map server configuration so that this map is the one loaded:

**start_map_server.launch**

In [ ]:
<launch>
    <node name="map_server" pkg="map_server" type="map_server" args="$(find sumit_xl_tools)/maps/new_clean_map.yaml"/>
</launch>

Now the map server will load the new map into the param server and it should be seen in RVIZ. When you launch the map server you should see something like this:

<img src="img/new_clean_map_rviz_nomaptf.png" width="600"></img>

As you can se the map appears in a default position, because there is no transform from the **/map** frame. Why? Because you dont have the AMCL which conected the **map** frame to the odom. So the next step is to publish the necessary TFS that position each element where it should, based in the GPS data.

### Get the position of the robot based on GPS,Odometry and IMU data

In indoor navigation you had as origin of the TF tree the **/map** frame. And from there you positioned the **odom** frame of the robot thanks to **AMCL** localization.<br>
Now there is NO AMCL, and therefore who will position the robot in the map?<br>
More over, who will position now the **map**? Because it has to be positioned based on GPS coordinates.

And here is where the package <a href="http://docs.ros.org/kinetic/api/robot_localization/html/index.html">robot_localization</a> comes into play.<br>
This package allows you to localise a Robot based on infinite number of inputs. You can merge the data from AMCL, GPS, Vision.. To get the best localization through Kalman Filters.<br>
In this course we will only talk about the tool <a href="http://docs.ros.org/kinetic/api/robot_localization/html/integrating_gps.html">navsat_transform_node</a>.

The **navsat_transform_node** alows you to convert the GPS **latitude** and **longitude** into **XY** coordinates that can be represented in space. Here is the example of the launch file you will need to launch it:

**start_navsat.launch**

In [ ]:
<launch>
 <!-- -->
  <node pkg="robot_localization" type="navsat_transform_node" name="navsat_transform_node" respawn="true">

    <param name="magnetic_declination_radians" value="0"/>
    <param name="yaw_offset" value="0"/>
    <param name="zero_altitude" value="true"/>
    
    
    <param name="broadcast_utm_transform" value="false"/>
    <param name="publish_filtered_gps" value="false"/>

    
    <param name="use_odometry_yaw" value="false"/>
    <param name="wait_for_datum" value="false"/>
    

    <remap from="/imu/data" to="/imu/data" />
    <remap from="/gps/fix" to="/gps/fix" />
    <remap from="/odometry/filtered" to="/odom" />

  </node>

</launch>

Some element to comment on:

In [ ]:
<param name="magnetic_declination_radians" value="0"/>
<param name="yaw_offset" value="0"/>
<param name="zero_altitude" value="true"/>

These paraemeters are just to adjust some offsets you might have in your sensors IMU and so on. You also can consider Null the altitude.

In [ ]:
<param name="broadcast_utm_transform" value="false"/>
<param name="publish_filtered_gps" value="false"/>

* broadcast_utm_transform: This is in case you want to publish the static TF of the UTM ( Origin of the GPS system ) in the TF tree. 
* publish_filtered_gps: No mystery here, if you want it to publish an improved by odometry and IMU GPS data.

In [ ]:
<param name="use_odometry_yaw" value="false"/>
<param name="wait_for_datum" value="false"/>

* use_odometry_yaw: This is for when you have a very reliable odometry turning system. In case of robots outdoors , the turning data in odometry is reliable enough because they skid once in a while , specially in rough terrain, thats why its turned OFF here.
* wait_for_datum: This is to give the system directly the datum, what is considered as the origin of our GPS localization. Normally its the GPS that directly gives this data and thats why its normaly turned off. If you need to give it for soe reason, you will have to add in this launch the following param setting:

In [ ]:
<rosparam param="datum">[55.944904, -3.186693, 0.0, map, base_link]</rosparam>

Or through the service **set_datum service and using the robot_localization/SetDatum service message**.

In [ ]:
<remap from="/imu/data" to="/imu/data" />
<remap from="/gps/fix" to="/gps/fix" />
<remap from="/odometry/filtered" to="/odom" />

This is the most important parameters. These are the topics from which it gets the GPS, IMU and Odometry data. With it it generates the Pose equivalent.

Once you have this launch ready, when launched, it will publish into the topic **/odometry/gps**.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1 to move Haro</p><br>
rosrun your_package start_navsat.launch
</th>
</tr>
</table>

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #2 to move Haro</p><br>
rostopic info /odometry/gps
</th>
</tr>
</table>

As you can see here, the message type is **nav_msgs/Odometry**

In [ ]:
## nav_msgs/Odometry                                                                                                              
                                                                                                                                                             
std_msgs/Header header                                                                                                                                       
  uint32 seq                                                                                                                                                 
  time stamp                                                                                                                                                 
  string frame_id                                                                                                                                            
string child_frame_id                                                                                                                                        
geometry_msgs/PoseWithCovariance pose                                                                                                                        
  geometry_msgs/Pose pose                                                                                                                                    
    geometry_msgs/Point position                                                                                                                             
      float64 x                                                                                                                                              
      float64 y                                                                                                                                              
      float64 z                                                                                                                                              
    geometry_msgs/Quaternion orientation                                                                                                                     
      float64 x                                                                                                                                              
      float64 y                                                                                                                                              
      float64 z                                                                                                                                              
      float64 w                                                                                                                                              
  float64[36] covariance                                                                                                                                     
geometry_msgs/TwistWithCovariance twist                                                                                                                      
  geometry_msgs/Twist twist                                                                                                                                  
    geometry_msgs/Vector3 linear                                                                                                                             
      float64 x                                                                                                                                              
      float64 y                                                                                                                                              
      float64 z                                                                                                                                              
    geometry_msgs/Vector3 angular                                                                                                                            
      float64 x                                                                                                                                              
      float64 y                                                                                                                                              
      float64 z                                                                                                                                              
  float64[36] covariance

As you can see,its a standard Odometry message. This means that now you have the position of the Summit XL based only on its GPS, Odometry and IMU.<br>
So the obvious step now is convert this data into a TF that positions the robot in the correct spot.

### Create TF publishers for MAP and Odom frame

You will have to publish the position of the robot in from of TF, so that afterwards , the move_base planner can generate atransform from the robots location to the map location.<br>
So now you have to create a python script that publishes that TF.

**publish_world_to_odom_and_map.py**

In [ ]:
#!/usr/bin/env python 

import rospy
import tf
from nav_msgs.msg import Odometry

class GPSXY(object):
    def __init__(self):
        rospy.Subscriber("/odometry/gps", Odometry, self.callback)
        self.x = 0.0
        self.y= 0.0

    def callback(self,msg):
        self.x = msg.pose.pose.position.x
        self.y= msg.pose.pose.position.y
        


if __name__ == '__main__':
    rospy.init_node('utm_to_map_node')
    br = tf.TransformBroadcaster()
    rate = rospy.Rate(3.0)
        
    gps_pos = GPSXY()
    
    while not rospy.is_shutdown():
        br.sendTransform((gps_pos.x, gps_pos.y, 0.0),
                         (0.0, 0.0, 0.0, 1.0),
                         rospy.Time.now(),
                         "odom",
                         "gps_origin")
                         
        rate.sleep()

So here you now can publish the position of the robot based on its GPS.<br>
Note that you are publishing from **odom** to **gps_origin**.This is where you want to consider the local GPS origin. You could instead of using this, publish the UTM origin, but for practical reasons, its better to publish one that its in close range of your location.<br>
And where is this Origin? Well in this case you are going to set it to the latitude and logintude of the officies of <a href="http://www.robotnik.es/">Robotnik</a>. This is latitude = 39.5080331, and longitude = -0.4619816. You can get the conversion from address to lat-longitude and the otherway round in this <a href="https://www.gps-coordinates.net/">Google page</a>.

So now the next step is to position the **/map** frame relative to the **/gps_origin** frame. This way you will now have a TF connectig the Robot with the map in someway, which is what the move_base needs.<br>
Just update the previous **publish_world_to_odom_and_map.py** version to this new one.

NEW **publish_world_to_odom_and_map.py**

In [ ]:
#!/usr/bin/env python 

# Import geonav tranformation module
import geonav_transform.geonav_conversions as gc
reload(gc)
# Import AlvinXY transformation module
import alvinxy.alvinxy as axy
reload(axy)
import rospy
import tf
from nav_msgs.msg import Odometry


def get_xy_based_on_lat_long(lat,lon, name):
    # Define a local orgin, latitude and longitude in decimal degrees
    # Robotnik position at Fuente del Jarro
    olat = 39.5080331
    olon = -0.4619816
    
    xg2, yg2 = gc.ll2xy(lat,lon,olat,olon)
    utmy, utmx, utmzone = gc.LLtoUTM(lat,lon)
    xa,ya = axy.ll2xy(lat,lon,olat,olon)

    rospy.loginfo("#########  "+name+"  ###########")  
    rospy.loginfo("LAT COORDINATES ==>"+str(lat)+","+str(lon))  
    rospy.loginfo("COORDINATES ==>"+str(xg2)+","+str(yg2))
    rospy.loginfo("COORDINATES ==>"+str(xa)+","+str(ya))
    rospy.loginfo("COORDINATES ==>"+str(utmx)+","+str(utmy))

    return xg2, yg2

class GPSXY(object):
    def __init__(self):
        rospy.Subscriber("/odometry/gps", Odometry, self.callback)
        self.x = 0.0
        self.y= 0.0

    def callback(self,msg):
        
        self.x = msg.pose.pose.position.x
        self.y= msg.pose.pose.position.y
        


if __name__ == '__main__':
    rospy.init_node('utm_to_map_node')
    br = tf.TransformBroadcaster()
    rate = rospy.Rate(3.0)
    xg2, yg2 = get_xy_based_on_lat_long(lat=39.5080331,lon=-0.4619816, name="MAP")
    
    gps_pos = GPSXY()
    
    while not rospy.is_shutdown():
        br.sendTransform((gps_pos.x, gps_pos.y, 0.0),
                         (0.0, 0.0, 0.0, 1.0),
                         rospy.Time.now(),
                         "odom",
                         "gps_origin")
                         
        br.sendTransform((xg2, yg2, 0.0),
                         (0.0, 0.0, 0.0, 1.0),
                         rospy.Time.now(),
                         "map",
                         "gps_origin")

        rate.sleep()

Here you are using a new ROS package called <a href="http://wiki.ros.org/geonav_transform">geonav_transform</a>. This package gives you tools to work with all the tranformations from GPS to Cartesian space and to UTM space. More information about UTM <a href="https://en.wikipedia.org/wiki/Universal_Transverse_Mercator_coordinate_system">here</a>.

Comment on some code:

In [ ]:
br.sendTransform((xg2, yg2, 0.0),
                         (0.0, 0.0, 0.0, 1.0),
                         rospy.Time.now(),
                         "map",
                         "gps_origin")

This is the core. Here you are publishing the Tranform from the map frame to the already published gps_origin. This will position the map.

In [ ]:
xg2, yg2 = get_xy_based_on_lat_long(lat=39.5080331,lon=-0.4619816, name="MAP")

This function is the one responsible of converting latitude and longitude to the Cartesian. It sets as origin olat = 39.5080331 and olon = -0.4619816. But you could set it alse where. In this particular case the map is set exactly in the same place as the **gps_origin** but thats just because the map is considered to be generated there. In reality it will be probably different.

In [ ]:
def get_xy_based_on_lat_long(lat,lon, name):
    # Define a local orgin, latitude and longitude in decimal degrees
    # Robotnik position at Fuente del Jarro
    olat = 39.5080331
    olon = -0.4619816
    
    xg2, yg2 = gc.ll2xy(lat,lon,olat,olon)
    utmy, utmx, utmzone = gc.LLtoUTM(lat,lon)
    xa,ya = axy.ll2xy(lat,lon,olat,olon)

    rospy.loginfo("#########  "+name+"  ###########")  
    rospy.loginfo("LAT COORDINATES ==>"+str(lat)+","+str(lon))  
    rospy.loginfo("COORDINATES ==>"+str(xg2)+","+str(yg2))
    rospy.loginfo("COORDINATES ==>"+str(xa)+","+str(ya))
    rospy.loginfo("COORDINATES ==>"+str(utmx)+","+str(utmy))

    return xg2, yg2

You can see that there are other conversions made. One of them is utm, which would be if you desire to use UTM frame as refference.

<p style="background:#EE9023;color:white;">Exercise U2-1</p>

Remove the AMCL launch from your move_base launch, and add:
* The **start_navsat.launch** to start the publications of /odometry/gps
* The launch of the python script **publish_world_to_odom_and_map.py** to publish the TF of map and odom.
The objective is to get in RVIZ the same exact appearance as when AMCL was working.

<p style="background:#EE9023;color:white;">END Exercise U2-1</p>

Please Try to do it by yourself unless you get stuck or need some inspiration. You will learn much more if you fight for each exercise.

<img src="img/robotignite_logo_text.png"/>

So you should have got something similar to this:

<img src="img/summit_xl_manual_unit2_gps_localization2.gif" width="600"></img>

And you should get something similar to this when setting a pose to move to:

<img src="img/summit_xl_manual_unit2_gps_localization_pose.gif" width="600"></img>

In [ ]:
Tha launch file for the move base should be something similar to this:

In [ ]:
<?xml version="1.0"?>
<!-- NEW SUMIT XL NAVIGATION -->
<launch>

    <node pkg="my_sumit_xl_tools" type="publish_world_to_odom_and_map.py" respawn="false" name="utm_to_map_node" output="screen"/>

    <!-- Start GPS to XY publication of the robot position -->
    

    <include file="$(find my_sumit_xl_tools)/launch/start_navsat.launch" />

    
    <!-- Run the map server -->
    <include file="$(find my_sumit_xl_tools)/launch/start_map_server.launch" />

    <remap from="cmd_vel" to="/move_base/cmd_vel" />
    
    <arg name="no_static_map" default="false"/>
    
    <arg name="base_global_planner" default="navfn/NavfnROS"/>
    <arg name="base_local_planner" default="dwa_local_planner/DWAPlannerROS"/>
    <!-- <arg name="base_local_planner" default="base_local_planner/TrajectoryPlannerROS"/> -->
    
    <node pkg="move_base" type="move_base" respawn="false" name="move_base" output="screen">
        
        <param name="base_global_planner" value="$(arg base_global_planner)"/>
        <param name="base_local_planner" value="$(arg base_local_planner)"/>  
        <rosparam file="$(find my_sumit_xl_tools)/new_config/planner.yaml" command="load"/>
        
        <!-- observation sources located in costmap_common.yaml -->
        <rosparam file="$(find my_sumit_xl_tools)/new_config/costmap_common.yaml" command="load" ns="global_costmap" />
        <rosparam file="$(find my_sumit_xl_tools)/new_config/costmap_common.yaml" command="load" ns="local_costmap" />
        
        <!-- local costmap, needs size -->
        <rosparam file="$(find my_sumit_xl_tools)/new_config/costmap_local.yaml" command="load" ns="local_costmap" />
        <param name="local_costmap/width" value="5.0"/>
        <param name="local_costmap/height" value="5.0"/>
        
        <!-- static global costmap, static map provides size -->
        <rosparam file="$(find my_sumit_xl_tools)/new_config/costmap_global_static.yaml" command="load" ns="global_costmap" unless="$(arg no_static_map)"/>
        
        <!-- global costmap with laser, for odom_navigation_demo -->
        <rosparam file="$(find my_sumit_xl_tools)/new_config/costmap_global_laser.yaml" command="load" ns="global_costmap" if="$(arg no_static_map)"/>
        <param name="global_costmap/width" value="100.0" if="$(arg no_static_map)"/>
        <param name="global_costmap/height" value="100.0" if="$(arg no_static_map)"/>
    </node>

</launch>


One Important improvement is the fact that because GPS has such a high level of error, you will have to change the tolerance to error of the move_base system. THats why you have to eddit the file **planner.yaml** and change the following part:

In [ ]:
  # Goal Tolerance Parameters  
  yaw_goal_tolerance: 0.1
  xy_goal_tolerance: 3.0
  # xy_goal_tolerance: 0.2
  latch_xy_goal_tolerance: false

The xy_goal_tolerance for planar position. The yaw should be quite precise because it comes from the IMU.

### Represent the MAP position in RVIZ

Because you have now GPS data, you can represent the robot in the real place where the GPS is positioning it. Getting results like:

<img src="img/summit_xl_instruction_manual_unit2_gpsmap1.png" width="600"></img>
<img src="img/summit_xl_instruction_manual_unit2_gpsmap2.png" width="600"></img>

But this is not what you should see, because if you retrieve the address from the <a href="https://www.gps-coordinates.net/">Google page</a> based on the origin latitude and longitude (olat = 39.5080331 olon = -0.4619816), you should get a map similar to the one shown in Google maps:

<img src="img/summit_xl_unit2_googlepam_robotnik.png" width="600"></img>

So to get this in RVIZ you have to:
* Represent the GPS Current Position in RVIZ using a plugin: This plugin allows you to position the summit XL based in the GPS data published in /gps/fix topic in the corresponding real place on Earth https://github.com/gareth-cross/rviz_satellite.<br>
    You can copy a rviz file that is already set for you in The path is **/opt/ros/indigo/share/sumit_xl_tools/rviz_config/gps.rviz**<br>
* You can set it manually as follows:
    ![Camera Icon Image](img/summit_xl_unit2_add_mapbox.png)<br>
    Then set the topic where the GPS data is published (For example:  /gps/fix ) , the RobotFrame ( for example /odom )and set the URL where to fetch the MapTiles. This URL has to have the following structure:<br>
    http://a.tiles.mapbox.com/v4/mapbox.satellite/{z}/{x}/{y}.jpg?access_token=YOUR_ACCESS_TOKEN<br>
    The Only thing that has to be changed is the YOUR_ACCESS_TOKEN which its given by MapBox https://www.mapbox.com/install/js/cdn-add/.
    

If everythin went well , RVIZ should show something like this:
![RViz Gps General](img/summit_xl_unit2_gpsimage_rviz.png)<br>
![RViz Gps General](img/summit_xl_unit2_gpsimage_rviz2.png)<br>

<p style="background:#EE9023;color:white;">Exercise U2-2</p>

Try different settings for the RVIZ **AerialMapDisplay**, see how it might change its visualization.

<p style="background:#EE9023;color:white;">END Exercise U2-2</p>

<p style="background:#EE9023;color:white;">Exercise U2-3</p>

Create now a program that moves the robot to three different GPS positions in space. Steps to follow:
* Create a GPS to Pose conversor: This can be accomplished copying what you have already done for the map TF publication.
* Create a client of move_base : This client will be the one in charge of sending to the move_base server the poses. Make a system that allows it to know when it has reached its destination.
You migh have to move by keyboard the robot until it gets out of the house, depending on the value syou set in the planner. If it can go through doors, then no need for this step.

<p style="background:#EE9023;color:white;">END Exercise U2-3</p>

Please Try to do it by yourself unless you get stuck or need some inspiration. You will learn much more if you fight for each exercise.

<img src="img/robotignite_logo_text.png"/>

Here you have an example of move_base client that migh come in handy.

**move_base_client.py**

In [ ]:
#! /usr/bin/env python

import rospy
import time
import actionlib
from move_base_msgs.msg import MoveBaseAction, MoveBaseGoal, MoveBaseResult, MoveBaseFeedback

"""
class SimpleGoalState:
    PENDING = 0
    ACTIVE = 1
    DONE = 2
    WARN = 3
    ERROR = 4

"""
# We create some constants with the corresponing vaules from the SimpleGoalState class
PENDING = 0
ACTIVE = 1
DONE = 2
WARN = 3
ERROR = 4

"""
/move_base/goal
### PYTHON MESSAGE

rosmsg show move_base_msgs/MoveBaseGoal                                                          
geometry_msgs/PoseStamped target_pose                                                                                          
  std_msgs/Header header                                                                                                       
    uint32 seq                                                                                                                 
    time stamp                                                                                                                 
    string frame_id                                                                                                            
  geometry_msgs/Pose pose                                                                                                      
    geometry_msgs/Point position                                                                                               
      float64 x                                                                                                                
      float64 y                                                                                                                
      float64 z                                                                                                                
    geometry_msgs/Quaternion orientation                                                                                       
      float64 x                                                                                                                
      float64 y                                                                                                                
      float64 z                                                                                                                
      float64 w                                                                                                                
               


/move_base/cancel                                                                                                                                                         
/move_base/cmd_vel                                                                                                                                                        
/move_base/current_goal                                                                                                                                                   
/move_base/feedback
"""

# definition of the feedback callback. This will be called when feedback
# is received from the action server
# it just prints a message indicating a new message has been received
def feedback_callback(feedback):
    rospy.loginfo(str(feedback))

# initializes the action client node
rospy.init_node('move_base_gps_node')

action_server_name = '/move_base'
client = actionlib.SimpleActionClient(action_server_name, MoveBaseAction)

# waits until the action server is up and running
rospy.loginfo('Waiting for action Server '+action_server_name)
client.wait_for_server()
rospy.loginfo('Action Server Found...'+action_server_name)

# creates a goal to send to the action server
goal = MoveBaseGoal()

goal.target_pose.header.frame_id = "/map"
goal.target_pose.header.stamp    = rospy.get_rostime()
goal.target_pose.pose.position.x = 0.0
goal.target_pose.pose.orientation.z = 0.0
goal.target_pose.pose.orientation.w = 1.0

client.send_goal(goal, feedback_cb=feedback_callback)


# You can access the SimpleAction Variable "simple_state", that will be 1 if active, and 2 when finished.
#Its a variable, better use a function like get_state.
#state = client.simple_state
# state_result will give the FINAL STATE. Will be 1 when Active, and 2 if NO ERROR, 3 If Any Warning, and 3 if ERROR
state_result = client.get_state()

rate = rospy.Rate(1)

rospy.loginfo("state_result: "+str(state_result))

while state_result < DONE:
    rospy.loginfo("Doing Stuff while waiting for the Server to give a result....")
    rate.sleep()
    state_result = client.get_state()
    rospy.loginfo("state_result: "+str(state_result))
    
rospy.loginfo("[Result] State: "+str(state_result))
if state_result == ERROR:
    rospy.logerr("Something went wrong in the Server Side")
if state_result == WARN:
    rospy.logwarn("There is a warning in the Server Side")

#rospy.loginfo("[Result] State: "+str(client.get_result()))

## Congratulations! You are now able to navigate with GPS to unmapped places.